In [ ]:
import subprocess
from subprocess import Popen
import pandas as pd
import numpy as np
import glob
import sys
from pathlib import Path

## Obtain bug prediction scores

In [ ]:
bugs = pd.read_csv(r'C:/Users/migue/Downloads/ruben-demo-code-lifetime-data.csv',  lineterminator='\n')

In [ ]:
#bugs = pd.read_csv('../data/generated/code-lifetime-data.csv.gz', compression='gzip',error_bad_lines=False,sep=' ')


In [ ]:
bugs.shape

In [ ]:
bugs.head()

In [ ]:
bugs['bug_fix'].unique()

In [ ]:
bugs['bug_id'].unique()

In [ ]:
groupdf = bugs.groupby(by=["bug_id",'line_number','file_path'])

In [ ]:
groupdf.head()

In [ ]:
import math

bugScore = pd.DataFrame()


j = 0
for i in groupdf:
        
    #print(i[1].shape)
    
    if(len(i[1]['author_commit_date']) == 1):
        
        i[1]['author_commit_date'] = i[1].iloc[0]['author_commit_date'] / i[1].iloc[0]['author_commit_date']
    
    else:
        
        i[1]['author_commit_date'] = (i[1]['author_commit_date'] - i[1]['author_commit_date'].min()) / (i[1]['author_commit_date'].max() - i[1]['author_commit_date'].min())
        
    
    i[1]['author_commit_date'] = 1/ (1 + math.e**(-12* i[1]['author_commit_date'] + 12))
   
    
    sumdf = i[1].groupby(by=["bug_fix"]).agg({'author_commit_date' : ['sum']})
    
    if sumdf.size == 1:
        
        twrrevisions = float((sumdf.iloc[0])['author_commit_date'])
        Bc = 0.35 * twrrevisions
        #print(Bc)
        defectC = 1 -math.e**(-Bc)
        #print(defectC)
    else :
        
        twrRevisions = float((sumdf.iloc[0])['author_commit_date'])
        twrFixRevisions = float((sumdf.iloc[1])['author_commit_date'])
        Bc = 0.35 * (twrRevisions + twrFixRevisions) + 0.65 * twrFixRevisions
        #print(Bc)
        defectC = 1 -math.e**(-Bc)
        #print(defectC)
    
    bug_id = int(i[0][0])
    line = int(i[0][1])
    #print(bug_id)
    #print(line)
    bugScore.at[j,'bug_probability'] = defectC
    bugScore.at[j,'bug_id'] = bug_id
    bugScore.at[j,'line_number'] = line
    bugScore.at[j,'fix_commit_hash'] = i[1].iloc[0]['fix_commit_hash']
    bugScore.at[j,'project_full_name'] = i[1].iloc[0]['project_full_name']
    bugScore.at[j,'bug_type'] = i[1].iloc[0]['bug_type']
    bugScore.at[j,'file_path'] =  i[0][2]
    
    j= j + 1
 
        
        


In [ ]:
#bugScore.to_csv('teste.csv',index=False)

In [ ]:
bugScore.head()

In [ ]:
bugScore.shape

In [ ]:
bugScore['bug_probability'].unique()

In [ ]:
bugScore.head(5)

In [ ]:
def removeZero(x):
    
    return x.split('.')[0]

In [ ]:
bugs2 = buggy_elements_df = pd.read_csv('../data/generated/buggy-code-lifetime-data.csv', sep=',')
bugs2 = bugs2.rename(columns={"buggy_line_number": "line_number"})
bugs2['bug_id'].unique()
bugs2['bug_id']=bugs2['bug_id'].astype(str)
bugs2['line_number']=bugs2['line_number'].astype(str)

In [ ]:
groupid = bugScore.groupby(by=["bug_id"])

In [ ]:
RQ3_df = pd.DataFrame()
for k, gp in groupid:
    print('key=' + str(k))
    gp['ranking_position'] = ''
    gp = gp.sort_values(by=['bug_probability'],ascending=False)
    gp.reset_index(inplace = True, drop = True)
    
    length = gp.shape[0]
    counter = 0
    for i in range(length):
        
        if(i == 0):
            gp.at[i,'ranking_position']= counter + 1
            counter = counter + 1
        elif((gp.iloc[i]['bug_probability']) == (gp.iloc[i-1]['bug_probability'])):
            gp.at[i,'ranking_position']= gp.iloc[i-1]['ranking_position']
        else:
            gp.at[i,'ranking_position']= counter + 1 
            counter = counter + 1
    
    
    gp['bug_id']=gp['bug_id'].astype(str)
    gp['line_number']=gp['line_number'].astype(str)
    gp['ranking_position']=gp['ranking_position'].astype(int)
    
    gp['bug_id'] = gp['bug_id'].apply(removeZero)
    gp['line_number'] = gp['line_number'].apply(removeZero)
    
    merge_df = pd.merge(gp,bugs2, on=["bug_id",'line_number'], how="inner")
    RQ3_df = RQ3_df.append(merge_df, ignore_index=True)
    #print(merge_df)
    #print(gp)
    
    

In [ ]:
#bugs2 = bugs2.drop_duplicates(subset=["bug_id", "line_number",'buggy_file_path'], keep='first')


In [ ]:
#bugScore.dtypes

In [ ]:
RQ3_df.head()

In [ ]:
RQ3_df.shape

In [ ]:
final_df = RQ3_df.groupby(by=["buggy_component"])

In [ ]:
RQ3_df = pd.DataFrame()

j = 0

for i in final_df:
    
    print(i[0])
    mean = int(round(i[1]['ranking_position'].mean(),0))
    median = int(round(i[1]['ranking_position'].median(),0))
    first = int(round(i[1].iloc[0]['ranking_position']))
    last = int(round(i[1].iloc[-1]['ranking_position']))
    top1 = sum(i[1]['ranking_position'] <= 1)
    top10 = sum(i[1]['ranking_position'] <= 10)
    top200 = sum(i[1]['ranking_position'] <= 200)
    print(mean)
    print(median)
    print(first)
    print(last)
    print(top1)
    print(top10)
    print(top200)
    
    RQ3_df.at[j,'buggy_component'] = i[0]
    RQ3_df.at[j,'mean'] =  mean
    RQ3_df.at[j,'median'] = median
    RQ3_df.at[j,'first'] = first
    RQ3_df.at[j,'last'] = last
    RQ3_df.at[j,'top1'] = top1
    RQ3_df.at[j,'top10'] = top10
    RQ3_df.at[j,'top200'] = top200
    
    j = j + 1
    
    
    
    


In [ ]:
RQ3_df.head(30)

Top1, Top10 and Top200 being the number of times a given buggy code element was found within topX in every ranking(rankings were grouped by bug_id hence more than 1 ranking. Previously we had only 1 ranking) 


In [ ]:
#bugs.to_csv('..\data\RQ3_Data.csv', index=False, encoding="utf8" )